## ALGO DEEP DIVE:
Alternating Least Square is a matrix factorisation algorithm implemented in Apache Spark ML and built for large-scale collaborative filtering problems.

In [0]:
%pip install recommenders
%pip install datetime

Python interpreter will be restarted.
Collecting recommenders
 Using cached recommenders-1.1.0-py3-none-manylinux1_x86_64.whl (335 kB)
Collecting memory-profiler<1,>=0.54.0
 Using cached memory_profiler-0.60.0-py3-none-any.whl
Collecting cornac<2,>=1.1.2
 Using cached cornac-1.14.2-cp38-cp38-manylinux1_x86_64.whl (14.4 MB)
Requirement already satisfied: transformers<5,>=2.5.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (4.15.0)
Requirement already satisfied: tqdm<5,>=4.31.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (4.59.0)
Requirement already satisfied: seaborn<1,>=0.8.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.11.1)
Requirement already satisfied: jinja2<3.1,>=2 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (2.11.2)
Collecting category-encoders<2,>=1.3.0
 Using cached category_encoders-1.3.0-py2.py3-none-any.whl (61 kB)
Requirement already satisfied: pyyaml<6,>=5.4.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (5.4.1)
Requirement already satisfied: pandas<2,>1.0.3 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (1.1.5)
Collecting bottleneck<2,>=1.2.1
 Using cached Bottleneck-1.3.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_24_x86_64.whl (331 kB)
Requirement already satisfied: requests<3,>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (2.25.1)
Requirement already satisfied: matplotlib<4,>=2.2.2 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.4.2)
Requirement already satisfied: nltk<4,>=3.4 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.6.1)
Requirement already satisfied: scipy<2,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (1.5.2)
Collecting scikit-surprise>=1.0.6
 Using cached scikit_surprise-1.1.1-cp38-cp38-linux_x86_64.whl
Requirement already satisfied: lightgbm>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.2.1)
Collecting retrying>=1.3.3
 Using cached retrying-1.3.3-py3-none-any.whl
Requirement already satisfied: numba<1,>=0.38.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.53.1)
Collecting lightfm<2,>=1.15
 Using cached lightfm-1.16-cp38-cp38-linux_x86_64.whl
Collecting pandera[strategies]>=0.6.5
 Using cached pandera-0.10.1-py3-none-any.whl (197 kB)
Requirement already satisfied: numpy>=1.19 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (1.19.5)
Requirement already satisfied: scikit-learn<1.0.3,>=0.22.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.22.2.post1)
Requirement already satisfied: statsmodels>=0.6.1 in /databricks/python3/lib/python3.8/site-packages (from category-encoders<2,>=1.3.0->recommenders) (0.11.1)
Requirement already satisfied: patsy>=0.4.1 in /databricks/python3/lib/python3.8/site-packages (from category-encoders<2,>=1.3.0->recommenders) (0.5.1)
Collecting powerlaw
 Using cached powerlaw-1.5-py3-none-any.whl (24 kB)
Requirement already satisfied: MarkupSafe>=0.23 in /databricks/python3/lib/python3.8/site-packages (from jinja2<3.1,>=2->recommenders) (2.0.1)
Requirement already satisfied: wheel in /databricks/python3/lib/python3.8/site-packages (from lightgbm>=2.2.1->recommenders) (0.36.2)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=2.2.2->recommenders) (1.3.1)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=2.2.2->recommenders) (0.10.0)
Requirement already satisfied: pillow>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=2.2.2->recommenders) (8.2.0)
Requirement already satisfied: pyparsing>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib<4,>=2.2.2->recommenders) (2.4.7)
Requirement already satis

In [0]:
# set the environment path to find Recommenders
import sys
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
Spark version: 3.2.1

In [0]:
df = spark.read.format("csv").options(header='true', delimiter = ',').load("abfss://.../mldata/MoviesDataRecommendation/ratings.csv")
df.show()
#import pandas as pd
#data=pd.read_csv("/dbfs/FileStore/df_ratings.csv", header='infer')
## Convert Pandas dataframe to spark DataFrame
#df = spark.createDataFrame(data)


# top k items to recommend
TOP_K = 10 
COL_USER = "userId"
COL_ITEM = "movieId"
COL_RATING = "rating"
COL_PREDICTION = "rating"
COL_TIMESTAMP = "timestamp"

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 1| 296| 5.0|1147880044|
 1| 306| 3.5|1147868817|
 1| 307| 5.0|1147868828|
 1| 665| 5.0|1147878820|
 1| 899| 3.5|1147868510|
 1| 1088| 4.0|1147868495|
 1| 1175| 3.5|1147868826|
 1| 1217| 3.5|1147878326|
 1| 1237| 5.0|1147868839|
 1| 1250| 4.0|1147868414|
 1| 1260| 3.5|1147877857|
 1| 1653| 4.0|1147868097|
 1| 2011| 2.5|1147868079|
 1| 2012| 2.5|1147868068|
 1| 2068| 2.5|1147869044|
 1| 2161| 3.5|1147868609|
 1| 2351| 4.5|1147877957|
 1| 2573| 4.0|1147878923|
 1| 2632| 5.0|1147878248|
 1| 2692| 5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows

In [0]:
# BinaryType: binary
# BooleanType: boolean
# ByteType: tinyint
# DateType: date
# DecimalType: decimal(10,0)
# DoubleType: double
# FloatType: float
# IntegerType: int
# LongType: bigint
# ShortType: smallint
# StringType: string
# TimestampType: timestamp
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import TimestampType
df = df.withColumn("userId", df["userId"].cast("int"))
df = df.withColumn("movieId", df["movieId"].cast("int"))
df = df.withColumn("rating", df["rating"].cast("double"))
df = df.withColumn("timestamp", df["timestamp"].cast("timestamp"))

In [0]:
''' 1.Random split
Random split simply takes in a data set and outputs the splits of the data, given the split ratios.
'''
train, test = spark_random_split(df, ratio=0.70, seed=123)
print("Size of Train: ",train.cache().count())
print("Size of Validate: ",test.cache().count())

Size of Train: 17502121
Size of Validate: 7497974

In [0]:
#Modelling
'''
Parameters:
- numBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
- rank is the number of latent factors in the model (defaults to 10).
- maxIter is the maximum number of iterations to run (defaults to 10).
- regParam specifies the regularization parameter in ALS (defaults to 1.0).
- implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
- alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).
- nonnegative specifies whether or not to use nonnegative constraints for least squares (defaults to false).
'''

header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING,
}


als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)
model = als.fit(train)


#Scoring
#In the movie recommendation use case, recommending movies that have been rated by the users do not make sense. Therefore, the rated movies are removed from the recommended items.In order to achieve this, we recommend all movies to all users, and then remove the user-movie pairs that exist in the training dataset.

# Get the cross join of all user-item pairs and score them.
users = train.select(COL_USER).distinct()
items = train.select(COL_ITEM).distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item) #scoring using the model trained

# Remove seen items.
spark = start_or_get_spark("ALS PySpark", memory="16g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train.alias("train"),
    (dfs_pred[COL_USER] == train[COL_USER]) & (dfs_pred[COL_ITEM] == train[COL_ITEM]),
    how='outer'
)

top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train[f"train.{COL_RATING}"].isNull()) \
    .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

top_all.cache().count()
top_all.show()

+------+-------+----------+
userId|movieId|prediction|
+------+-------+----------+
 1| 587| 2.6998122|
 1| 869| 2.9099126|
 1| 1208| 4.0218315|
 1| 1348| 3.5810237|
 1| 1357| 3.883794|
 1| 1677| 2.7238045|
 1| 1702| 1.7985499|
 1| 1720| 1.4157087|
 1| 1892| 3.0486994|
 1| 2086| 2.6992016|
 1| 2202| 3.520183|
 1| 2324| 4.223861|
 1| 2483| 3.3862746|
 1| 2545| 3.8411288|
 1| 2667| 1.291569|
 1| 2870| 3.2438471|
 1| 3304| 3.8208761|
 1| 3452| 2.5336146|
 1| 3468| 3.8727543|
 1| 3477| 3.4407997|
+------+-------+----------+
only showing top 20 rows

In [0]:
#train.where((train.userId==1) & (train.movieId==1348)).show()
#These are already avaialble in data as rated, so excluded from the top_all. Only predicted values of rating for user/movies not avaialble in training set

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 296| 5.0| null|
 1| 306| 3.5| null|
 1| 307| 5.0| null|
 1| 665| 5.0| null|
 1| 899| 3.5| null|
 1| 1088| 4.0| null|
 1| 1175| 3.5| null|
 1| 1217| 3.5| null|
 1| 1237| 5.0| null|
 1| 1250| 4.0| null|
 1| 1260| 3.5| null|
 1| 1653| 4.0| null|
 1| 2011| 2.5| null|
 1| 2012| 2.5| null|
 1| 2068| 2.5| null|
 1| 2161| 3.5| null|
 1| 2351| 4.5| null|
 1| 2573| 4.0| null|
 1| 2632| 5.0| null|
 1| 2692| 5.0| null|
+------+-------+------+---------+
only showing top 20 rows

In [0]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction", 
                                    relevancy_method="top_k")
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.000000
NDCG:	0.000024
Precision@K:	0.000022
Recall@K:	0.000002

In [0]:
# Generate predicted ratings.

#prediction v/s original rating in test dataframe
prediction = model.transform(test)
prediction.cache().show()

rating_eval = SparkRatingEvaluation(test, prediction, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')



+------+-------+------+---------+----------+
userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
 3411| 148| 3.0| null| 3.1010756|
 31376| 148| 2.0| null| 2.487941|
138552| 148| 4.0| null| 3.3528645|
 73925| 148| 2.5| null| 2.8853617|
 18628| 148| 2.0| null| 2.5358372|
 91345| 148| 3.0| null| 2.5081975|
136364| 148| 4.0| null| 3.1493373|
110365| 148| 4.0| null| 2.788095|
153108| 148| 1.0| null| 2.5116093|
122236| 148| 2.5| null| 2.4000401|
140907| 148| 4.0| null| 2.5808144|
153916| 148| 3.0| null| 2.9606023|
 25467| 148| 3.0| null| 2.400282|
 32268| 148| 4.0| null| 2.9746733|
 38679| 148| 3.0| null| 2.53785|
 99684| 148| 3.0| null| 2.9070284|
108767| 148| 3.0| null| 2.6903741|
157307| 148| 3.0| null| 2.7582307|
 66358| 148| 3.5| null| 2.92726|
 81091| 148| 1.0| null| 2.54278|
+------+-------+------+---------+----------+
only showing top 20 rows

Model:	ALS rating prediction
RMSE:	0.791485
MAE:	0.606727
Explained variance:	0.450670
R squared:	0.443385